# Adversarial example using FGSM(Needs Dataset on local machine)

<b>We have used FGSM generation example code from the below example and modified it our needs for making a Techincal Report.</b><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/adversarial_fgsm.ipynb">View source on GitHub</a>
</br>
</br>
<b>The stored excel files for differnet models and different datasets are used for creation of graphs represented in Techincal review</b></br></br>
<b>The dataset consists of 1500 images downloaded from imageCV</b> <a target="_blank" href="https://images.cv">imagenet</a></br></br>
<b>All the models are trained as using imagenet</b> <a target="_blank" href="https://image-net.org/">imagenet</a></br></br>

In [96]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import xlwt

In [97]:
mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

Helper function to preprocess the image so that it can be inputted into neural network or model


In [98]:
# Helper function to preprocess the image so that it can be inputted into neural network or model
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (224, 224))
  image = tf.keras.applications.densenet.preprocess_input(image)
  image = image[None, ...]
  return image

Helper function to extract labels from probability vector

In [99]:
# Helper function to extract labels from probability vector
def get_imagenet_label(probs):
  #print(decode_predictions(probs, top=1)[0][0])
  return decode_predictions(probs, top=1)[0][0]

In [100]:
def display_images(image, description):
  _, label, confidence = get_imagenet_label(pretrained_model.predict(image))
  #Commenting out the code image plotting code as this code is created or modified to manage huge amout of datasets.
  #plt.figure()
  #plt.imshow(image[0]*0.5+0.5)
  #plt.title('{} \n {} : {:.2f}% Confidence'.format(description,label, confidence*100))

  return label,confidence

In [101]:
def get_index(input_image):
    #calculating the prediction probabilities across all the labels of imagenet
    label_probalities = pretrained_model(input_image)
    #extracting the array of predictions from tensorflow object using numpy method of tensorflow
    label_array=label_probalities.numpy()[0]
    #returing the index where the prediction value is higher than all the others
    return label_array.argmax()

In [102]:
  
def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = pretrained_model(input_image)
    
    #calculating Crossentropyloss using CategoricalCrossentropy
    
    loss = loss_object(input_label, prediction)
    print(loss.numpy())
  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  # Get the sign of the gradients to create the perturbation
  # sign function outputs -1 if x < 0, 0 if x==0, 1 if x > 0
  signed_grad = tf.sign(gradient)
  return signed_grad

In [103]:
#initiating new workbook and writing the coloumn names
Workbook=xlwt.Workbook()
Worksheet=Workbook.add_sheet("sheet_name")
Worksheet.write(0,0,"Image_name")
Worksheet.write(0,1,"Epsilon 0")
Worksheet.write(0,2,"Epsilon 0.01")
Worksheet.write(0,3,"Epsilon 0.1")
Worksheet.write(0,4,"Epsilon 0.15")

#initiating the model in this case ResNet152V2 , we have tested this with densenet and mobilenetV2 also 
# model wieghts or pre-trained on imagenet
pretrained_model = tf.keras.applications.ResNet152V2(include_top=True,
                                                     weights='imagenet')

#making the model non trainable as our aim is to fool the trained model
pretrained_model.trainable = False

In [104]:
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

In [105]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

In [106]:
os.chdir('path\\to\\dataset\\folder\\author\\sudesh')
cwd=os.getcwd() #getting current working directory as we will be handling huge amoout of datasets.
# listing the image file names and counting the number of images.
images=os.listdir() 
num_of_images=len(images) 

# for every image in the dataset we are executing the below code.

for k in range(0,num_of_images):
  
  #reading the image and prepossing the image such that it can be submitted to the model
    
  image_raw = tf.io.read_file(cwd+"\\"+images[k])
  image = tf.image.decode_image(image_raw)
  image = preprocess(image)

  #submitting the image to the model and retreiving the predictions
    
  image_probs = pretrained_model.predict(image)
  _, image_class, class_confidence = get_imagenet_label(image_probs)
    
  #getting the label of the image.
  label_index = get_index(image)
  
  #   #creating tensorflow object based on label_index such that it can be submitted to create_adversarial_pattern method 
  #to generate perturbations

  label = tf.one_hot(label_index, image_probs.shape[-1])
  label = tf.reshape(label, (1, image_probs.shape[-1]))  
  perturbations = create_adversarial_pattern(image, label)

  #writing the image name to worksheet

  Worksheet.write(k+1,0,images[k])
    
  # array of epsilon values
  epsilons = [0, 0.01, 0.1, 0.15]
  descriptions = [('Epsilon = {:0.3f}'.format(eps) if eps else 'Input')
                  for eps in epsilons]
 
  for i, eps in enumerate(epsilons):
    
    #creating adversarial images by combining image with perturbations multiplied by eps
    adv_x = image + eps*perturbations
    adv_x = tf.clip_by_value(adv_x, -1, 1)
    
    #collecting the label detected and confidence of perturbated images.
    label_detected,confidence = display_images(adv_x, descriptions[i])
    Worksheet.write(k+1,i+1,label_detected+'('+str(confidence*100)+')')
    #writing it to xls file
Workbook.save('Filename.xls')

1.8391348
1.6290054
0.039658215
1.0032259
0.9822746
1.265406
0.037520386
0.5068579
0.5347562
2.154755
1.3662841
1.5268553
1.078107
1.9457966
1.4153956
0.90997565
1.8964714
0.10196557
0.33840504
1.2447226
0.39296958
0.18539178
1.8254468
0.14096335
0.81097025
0.18039332
1.1759006
0.06857697
0.13935249
0.18252741
0.9679614
0.46756902
0.024764229
0.1380076
0.029584816
1.8100718
0.7873336
0.2016861
0.16657792
0.8945836
1.3790927
0.012352276
1.8386296
0.21769834
1.1256003
1.6576155
0.798827
2.2557387
1.2775688
0.27406603
1.4574662
0.9892485
0.005343678
1.3257617
1.5505307
0.7492521
1.0562913
0.72136843
0.2651161
1.0251391
0.74972475
1.5810742
0.94107544
0.32034644
0.63972557
0.33476207
2.064329
0.33549687
2.0338442
1.3286324
1.4152932
0.6639339
0.3047828
1.3860083
0.04892544
1.4081632
0.7813827
0.6189529
1.630581
0.090497434
0.48989213
2.6324284
0.051381435
0.004594247
1.4712037
1.3112352
2.201413
0.8119795
0.6398325
1.4949344
0.0026062115
0.48907515
0.0039746347
0.6164445
1.5873662
0.585118

1.1565616
0.7680125
1.7495142


<b>The stored excel files for differnet models and different datasets are used for creation of graphs represented in Techincal review</b></br></br>
<b>The dataset consists of 1500 images downloaded from imageCV</b> <a target="_blank" href="https://images.cv">imagenet</a></br></br>
<b>All the models are trained as using imagenet</b> <a target="_blank" href="https://image-net.org/">imagenet</a></br></br>